## Spam mail分類を行う

In [36]:
%matplotlib inline

import libs.load_spam_data
import importlib
importlib.reload(libs.load_spam_data)

FILE_PATH = './spam_data/spam.csv'
train_tokes, train_labels, test_tokes, test_labels = libs.load_spam_data.load_spamd_data(FILE_PATH)




[nltk_data] Downloading package punkt to /Users/nisida/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nisida/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2.0.2
-------------
hum count   =  4825
supam count =  747
-------------
max_token_len =  171
[665, 202, 221, 666, 667, 668, 595, 477, 21, 669, 294, 670, 553]


{'name': 1, 'price':2} ->  all_token内の全ての単語にユニークなID（数字)をふる（Pythonの辞書で作成する）. -> コーパスと呼ぶ。

気になるpoint

token1 = [1,3, 6, 7] , token2 = [10, 13, 14, 15 16, 17]


In [ ]:
import nltk 

print(nltk.__version__)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import os

# データセットの準備
# ここでは、簡潔のためダミーデータを使用します。
# 実際には、スパムメール分類の公開データセットを使用してください。
# 例: KaggleのSMS Spam Collection datasetなど
data = {'text': ['Call me back urgent!!', 'Hello, how are you?', 'WIN a prize NOW!', 'Meeting at 3 PM.', 'Free money offer!', 'Please disregard.'],
        'label': ['spam', 'ham', 'spam', 'ham', 'spam', 'ham']}
df = pd.DataFrame(data)

# ラベルのエンコーディング
# 'spam' を 1 に、'ham' を 0 に変換
encoder = LabelEncoder()
df['label'] = encoder.fit_transform(df['label'])

# データの分割
X = df['text']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# テキストの前処理 (トークン化とパディング)

# 語彙サイズの設定
vocab_size = 1000
# 各シーケンスの最大長
max_len = 50

# Tokenizerの初期化と訓練データでのfitting
tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# テキストをシーケンスに変換
train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

# シーケンスのパディング
train_padded = pad_sequences(train_sequences, maxlen=max_len, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=max_len, padding='post', truncating='post')

# NumPy配列に変換 (TensorFlow/Kerasでの使用のため)
train_padded = np.array(train_padded)
test_padded = np.array(test_padded)
y_train = np.array(y_train)
y_test = np.array(y_test)

# モデルの構築
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, 16, input_length=max_len),
    keras.layers.GlobalAveragePooling1D(), # または Flatten(), LSTM(), Conv1D() など
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid') # スパムかハムかの二値分類のためsigmoid
])

# モデルのコンパイル
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# モデルのサマリーを表示
model.summary()

# モデルの学習
num_epochs = 30
history = model.fit(train_padded, y_train, epochs=num_epochs, validation_data=(test_padded, y_test), verbose=2)

# モデルの評価
loss, accuracy = model.evaluate(test_padded, y_test, verbose=2)
print(f"Test Accuracy: {accuracy}")

# 新しいメールでの予測例
sample_email = ["Claim your free gift now!", "Meeting moved to tomorrow."]
sample_sequences = tokenizer.texts_to_sequences(sample_email)
sample_padded = pad_sequences(sample_sequences, maxlen=max_len, padding='post', truncating='post')

predictions = model.predict(sample_padded)

# 予測結果の表示
print("\n--- 予測結果 ---")
for i, email in enumerate(sample_email):
    # シグモイド出力が0.5より大きければスパムと判断
    predicted_label = "spam" if predictions[i] > 0.5 else "ham"
    print(f"メール: \"{email}\" -> 予測: {predicted_label} ({predictions[i][0]:.4f})")